# 03: Aggregation Strategies Deep Dive

This notebook provides a concise overview of the main aggregation strategies available in our federated learning framework.

## Setup and Imports

In [ ]:
import sys
sys.path.append('..')

from core.strategies import create_strategy
import numpy as np
import torch
import flwr as fl

## Strategy Categories

Our framework supports three main strategy categories:

### 1. Basic Strategies
- **FedAvg**: Standard weighted averaging
- **FedAvgM**: FedAvg with server momentum
- **FedAdam**: Adaptive optimization

### 2. Robust Strategies (Byzantine-tolerant)
- **Krum**: Select most trustworthy client
- **TrimmedMean**: Remove outliers before averaging
- **Bulyan**: Multi-Krum with trimmed mean

### 3. Advanced Strategies
- **SCAFFOLD**: Variance reduction
- **FedProx**: Proximal term regularization
- **FedNova**: Normalized averaging

## Creating Strategies

In [ ]:
# Basic FedAvg strategy
fedavg_strategy = create_strategy('fedavg')
print(f"FedAvg strategy: {type(fedavg_strategy).__name__}")

# Robust Krum strategy
krum_strategy = create_strategy('krum', num_malicious_clients=2)
print(f"Krum strategy: {type(krum_strategy).__name__}")

# Advanced SCAFFOLD strategy
scaffold_strategy = create_strategy('scaffold')
print(f"SCAFFOLD strategy: {type(scaffold_strategy).__name__}")

## Strategy Use Cases

| Strategy | Best For | Key Parameters |
|----------|----------|----------------|
| FedAvg | Standard FL scenarios | `fraction_fit`, `fraction_evaluate` |
| Krum | Byzantine attacks | `num_malicious_clients` |
| TrimmedMean | Outlier handling | `beta` (trimming fraction) |
| SCAFFOLD | Non-IID data | None (automatic) |
| FedProx | System heterogeneity | `proximal_mu` |
| FedNova | Variable local steps | None (automatic) |

## Quick Experiment Comparison

In [ ]:
# Compare different strategies on the same setup
strategies_to_test = ['fedavg', 'fedprox', 'scaffold']

for strategy_name in strategies_to_test:
    strategy = create_strategy(strategy_name)
    print(f"\n{strategy_name.upper()}:")
    print(f"  Type: {type(strategy).__name__}")
    print(f"  Description: {strategy.__doc__.split('.')[0] if strategy.__doc__ else 'No description'}")

## Configuration Examples

In [ ]:
# Example configurations for different scenarios

# Scenario 1: Standard FL with good network
config_standard = {
    'strategy': 'fedavg',
    'fraction_fit': 0.3,
    'fraction_evaluate': 0.1
}

# Scenario 2: Adversarial environment
config_adversarial = {
    'strategy': 'krum',
    'num_malicious_clients': 2,
    'fraction_fit': 0.5
}

# Scenario 3: Non-IID data
config_noniid = {
    'strategy': 'scaffold',
    'fraction_fit': 0.4
}

print("Configuration examples created successfully!")

## Next Steps

- See `02_Basic_Federated_Learning_Experiment.ipynb` for running experiments
- Check the `attacks/` folder for adversarial scenarios
- Use `run_extensive_experiments.py` for comprehensive comparisons

**Key Takeaway**: Choose your aggregation strategy based on your specific FL scenario - standard FedAvg for typical cases, robust strategies for adversarial environments, and advanced strategies for challenging data distributions.